### 1. Imports

In [1]:
# 1. Imports
from dotenv import load_dotenv
load_dotenv()

import hashlib
import os
from pathlib import Path

from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document

from docling.document_converter import DocumentConverter

print("All Imports Successful")

All Imports Successful


### 2. Configurations

In [2]:
# 2. Configurations
DATA_DIR = "data"
CHROMA_DIR = "./chroma_financial_db"
COLLECTION_NAME = "financial_docs"
EMBEDDING_MODEL = "nomic-embed-text:latest"
BASE_URL = "http://localhost:11434"

### 3. Embeddings and Vector Store Set-up

In [3]:
# 3. Embeddings and Vector Store
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL, base_url=BASE_URL)

vector_store = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings,
    persist_directory=CHROMA_DIR
)

### 4. Metadata Extraction

In [ ]:
# Cell 4: Metadata Extraction
def extract_metadata_from_filename(filename: str) -> dict:
    """
        Extract metadata from filename.

        Expected format: {company} {doc_type} {quarter} {year}.pdf
        Examples:
        - amazon 10k 2024.pdf


        Returns:
            dict with company_name, doc_type, fiscal_year, fiscal_quarter
    """

    name = filename.replace('.pdf', '')

    parts = name.split()
    metadata = {}
    if len(parts) == 4:
        metadata["fiscal_quarter"] = parts[2]
        metadata["fiscal_year"] = int(parts[3])
    
    else:
        metadata["fiscal_quarter"] = None
        metadata["fiscal_year"] = int(parts[2])
    
    metadata["company_name"] = parts[0]
    metadata["doc_type"] = parts[1]

    return metadata


In [5]:
extract_metadata_from_filename('amazon 10-k 2023.pdf')
extract_metadata_from_filename('amazon 10-q q1 2024.pdf')

{'fiscal_quarter': 'q1',
 'fiscal_year': 2024,
 'company_name': 'amazon',
 'doc_type': '10-q'}

### 5. Extract text from each PDF page

In [ ]:
def extract_pdf_page(pdf_path):
    converter = DocumentConverter()

    result = converter.convert(pdf_path)

    page_break = "<!-- page break -->"

    markdown_text = result.document.export_to_markdown(page_break_placeholder=page_break)

    pages = markdown_text.split(page_break)

    return pages

In [7]:
"""Compute SHA-256 hash of file content."""
def compute_file_hash(file_path: str) -> str:
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

In [8]:
compute_file_hash('data/amazon/amazon 10-k 2023.pdf')

'6e5549c7b20b0fbc5f482397070a1e85cbf8643c801ff570903f52366b11154f'

### 6. Track the processed files

In [9]:
existing_docs = vector_store.get(where={"file_hash": {"$ne": ""}}, include=['metadatas'])
processed_hash = [m.get("file_hash") for m in existing_docs['metadatas'] if m.get("file_hash")]
processed_hash = set(processed_hash)
existing_docs

{'ids': ['c05dd385-e11f-48e6-81f7-c20732fbb3b8',
  '33574f81-beb6-489e-9f57-9cbc64da484e',
  'fd975e08-b815-4fdf-a269-8b8fe2585a45',
  '0a676c84-6b86-4a63-8e06-78d115c1dc6f',
  'a66e5016-7974-404a-85f7-0f6bff2dc851',
  'bc13cd54-80b8-478d-b759-d040bc1deea7',
  'ad003402-8f1d-408e-a342-949798f77005',
  '25c23c07-65ec-4a92-bf43-0fde1d9084aa',
  'd82989cf-ac31-4a9e-b968-2f5b1ae99c97',
  'd675ae37-57a5-473b-985d-f978baa59686',
  '0ff6a454-64ce-422c-a7e1-529481db02db',
  'ff5e7e68-f190-4e75-8b20-1599247aa09e',
  'c233e6df-7cf6-41af-8f95-8a8be2ca498c',
  '976d3209-1bdb-4e8d-ab88-719c77f3137e',
  'ea6d157e-1930-4a2c-a876-3d596e958537',
  '272ce3ff-eb26-4da8-85fb-864a28ac50d1',
  'fc805d28-ab08-429f-97be-c32b8e9821bc',
  '5c2f1b4b-cdfc-41fc-b5b9-525b64a84ec8',
  '4c3469d3-42f1-457c-946c-b68b99664056',
  '70b3a820-2b85-43e5-8d4c-3f93ce39aead',
  '5c0de30f-8d73-436b-8896-db547588ec10',
  '80c5f9ff-29fb-4760-ac84-88f11adccda0',
  '3ee2bfda-12d9-4ba4-a460-5b013d34806e',
  '3a25e2ad-8c97-4417-aaee-

### 7. Document Ingestion in Vector DB

In [23]:
# def ingest_docs_in_vectordb(pdf_path):
#     print(f"Processing: {pdf_path.name}")
#     file_hash = compute_file_hash(pdf_path)
#     if file_hash in processed_hash:
#         print(f"Already Processed: {pdf_path}")
#         return
    
#     pages = extract_pdf_page(pdf_path)

#     file_metadata = extract_metadata_from_filename(pdf_path.name)

#     processed_pages = []
#     for page_num, page_text in enumerate(pages, start=1):
#         metadata_dict = file_metadata.copy()
#         metadata_dict['page'] = page_num
#         metadata_dict["file_hash"] = file_hash
#         metadata_dict["source_file"] = pdf_path.name

#         doc = Document(page_content=page_text, metadata=metadata_dict)

#         processed_pages.append(doc)
#         print(len(page_text))
    
    # vector_store.add_documents(documents=processed_pages)


def ingest_docs_in_vectordb(pdf_path):
    print(f"Processing: {pdf_path.name}")
    file_hash = compute_file_hash(pdf_path)

    if file_hash in processed_hash:
        print(f"Already Processed: {pdf_path}")
        return
    
    pages = extract_pdf_page(pdf_path)
    file_metadata = extract_metadata_from_filename(pdf_path.name)

    processed_pages = []
    MAX_CHARS = 15000   

    for page_num, page_text in enumerate(pages, start=1):
        print("Original length:", len(page_text))

        if len(page_text) > MAX_CHARS:
            print(f"⚠ Truncating page {page_num} from {len(page_text)} chars")
            page_text = page_text[:MAX_CHARS]

        metadata_dict = file_metadata.copy()
        metadata_dict['page'] = page_num
        metadata_dict["file_hash"] = file_hash
        metadata_dict["source_file"] = pdf_path.name

        doc = Document(page_content=page_text, metadata=metadata_dict)
        processed_pages.append(doc)

    for doc in processed_pages:
        vector_store.add_documents([doc])
    

In [24]:
data_path = Path(DATA_DIR)
pdf_files = list(data_path.rglob("*.pdf"))
pdf_files

[PosixPath('data/apple/apple 10-k 2023.pdf'),
 PosixPath('data/apple/apple 10-q q2 2024.pdf'),
 PosixPath('data/apple/apple 10-k 2024.pdf'),
 PosixPath('data/apple/apple 8-k q4 2023.pdf'),
 PosixPath('data/apple/apple 10-q q4 2023.pdf'),
 PosixPath('data/apple/apple 10-q q1 2024.pdf'),
 PosixPath('data/amazon/amazon 10-q q1 2025.pdf'),
 PosixPath('data/amazon/amazon 10-q q1 2024.pdf'),
 PosixPath('data/amazon/amazon 10-k 2023.pdf'),
 PosixPath('data/amazon/amazon 10-q q2 2025.pdf'),
 PosixPath('data/amazon/amazon 10-k 2024.pdf'),
 PosixPath('data/amazon/amazon 10-q q2 2024.pdf'),
 PosixPath('data/google/google 10-q q1 2025.pdf'),
 PosixPath('data/google/google 10-k 2024.pdf'),
 PosixPath('data/google/google 10-k 2023.pdf'),
 PosixPath('data/google/google 10-q q2 2024.pdf')]

In [25]:
for pdf_path in pdf_files:
    ingest_docs_in_vectordb(pdf_path)

Processing: apple 10-k 2023.pdf
Already Processed: data/apple/apple 10-k 2023.pdf
Processing: apple 10-q q2 2024.pdf
Already Processed: data/apple/apple 10-q q2 2024.pdf
Processing: apple 10-k 2024.pdf
Already Processed: data/apple/apple 10-k 2024.pdf
Processing: apple 8-k q4 2023.pdf
Already Processed: data/apple/apple 8-k q4 2023.pdf
Processing: apple 10-q q4 2023.pdf
Already Processed: data/apple/apple 10-q q4 2023.pdf
Processing: apple 10-q q1 2024.pdf
Already Processed: data/apple/apple 10-q q1 2024.pdf
Processing: amazon 10-q q1 2025.pdf
Already Processed: data/amazon/amazon 10-q q1 2025.pdf
Processing: amazon 10-q q1 2024.pdf
Already Processed: data/amazon/amazon 10-q q1 2024.pdf
Processing: amazon 10-k 2023.pdf
Already Processed: data/amazon/amazon 10-k 2023.pdf
Processing: amazon 10-q q2 2025.pdf
Already Processed: data/amazon/amazon 10-q q2 2025.pdf
Processing: amazon 10-k 2024.pdf
Already Processed: data/amazon/amazon 10-k 2024.pdf
Processing: amazon 10-q q2 2024.pdf
Already

In [26]:
vector_store._collection.count()

1262

In [28]:
vector_store.get(where={"company_name": "amazon"}, limit=2)

{'ids': ['f1ef24bc-2b40-46cd-a7d7-d6e7fd05c62e',
  'f19d1f2e-2a21-4571-a917-f39440612522'],
 'embeddings': None,
 'documents': ["## UNITED STATES\n\n## SECURITIES AND EXCHANGE COMMISSION\n\nWashington, D.C. 20549\n\n\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\n\nFORM 10-Q\n\n\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\n\n(Mark One)\n\n- [x] ☒ 1934\n\n- [x] QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF\n\nFor the quarterly period ended March 31, 2025\n\nor\n\n- [ ] ☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the transition period from            to             .\n\nCommission File No. 000-22513\n\n\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\\_\n\n## AMAZON.COM, INC.\n\n(Exact name of registrant as specified i